### 댓글 분석과 예측

#### 분석 기법

- 나이브 베이즈 - TFid, Multi
- 감성분석 - Kosac and NB(nltk)
- 라벨링 - Kosac 사전 이용

#### 나이브 베이즈 
- TfidfVectorizer : 토큰화
- MultinomialNB : 나이브 베이즈 classify, predict

In [25]:
# 데이터 로드
article_df = pd.read_csv("tada_nb_comm_class.csv")
len(article_df)

1922

In [26]:
# 결측치 존재, class 선별 과정에서 공백 생김
article_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1922 entries, 0 to 1921
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   _id         1144 non-null   object 
 1   title       1144 non-null   object 
 2   date        1144 non-null   object 
 3   youtube_id  1132 non-null   object 
 4   comment     1144 non-null   object 
 5   like_num    1144 non-null   object 
 6   class       1144 non-null   float64
dtypes: float64(1), object(6)
memory usage: 105.2+ KB


In [27]:
# 결측치 제거
article_df = article_df.dropna(axis=0, how='any')
article_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132 entries, 0 to 1921
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   _id         1132 non-null   object 
 1   title       1132 non-null   object 
 2   date        1132 non-null   object 
 3   youtube_id  1132 non-null   object 
 4   comment     1132 non-null   object 
 5   like_num    1132 non-null   object 
 6   class       1132 non-null   float64
dtypes: float64(1), object(6)
memory usage: 70.8+ KB


In [28]:
# 중복제거1
test_data = list(set(article_df.comment))
len(test_data)

871

In [29]:
# 중복제거2, 감성 분석 라벨링 용
test_data_rm_du = article_df.drop_duplicates(['comment'])
len(test_data_rm_du)

871

In [30]:
# train, test 분리. 클래스 비율 고려하지 않음.
value_ =  np.random.rand(len(article_df)) < 0.9
test = article_df[~value_]
train = article_df[value_]
len(test), len(train)
test_data = list(test.comment)

In [31]:
# 데이터 전처리 : 데이터 셋 분리
X_train, X_test, y_train, y_test = train_test_split(
train.comment, train['class'], test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(915, 102, 915, 102)

In [32]:
# 모델 만들기
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.1)),
])

In [33]:
# 학습
model = clf.fit(X_train.values.astype("str"), y_train)

In [34]:
# 모델 성능 확인
y_pred = model.predict(X_test)

In [35]:
list(y_test[:5]), list(y_pred[:5])

([2.0, 3.0, 2.0, 1.0, 2.0], [2.0, 3.0, 2.0, 1.0, 2.0])

In [36]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        1.0       0.65      0.59      0.62        29
        2.0       0.73      0.83      0.77        52
        3.0       0.65      0.52      0.58        21

avg / total       0.69      0.70      0.69       102



#### 모델 사용하기

In [37]:
categories = {
    1: "타다",
    2: "택시",
    3: "미분류",

}

In [38]:
# test_data 적용
datas = {
    "content": test_data,
    "category_code": model.predict(test_data),
}
df = pd.DataFrame(datas)
df

,content,category_code
0,타다 1500대인데 기사가 12000명 이면 차한대에 기사가 10명정도 라는 말인데...,2.0
1,회사택시하세요 모든것이보장되어있는데 왜타다를선택했나요 뻔히불법이고결과가이럴줄몰랐나요?,2.0
2,불법콜택시 타다 면허ㆍ자격증 갖고하면 될껄 꽁짜로 쳐묵을려니까 문제지 총량제 ...,2.0
3,타다 잘 망했다 ~~~ 타다 만세,2.0
4,국회가 너무 늙었다..\n\n 🙆💬OMG!💨\n 👕\n 👖\n👟👟,3.0
...,...,...
110,공산국가??타다 금지법????,1.0
111,타다 베이직은 종료하고. 공항이랑 프라이빗렌트 서비스는 합법이니 계속하겠는데요,1.0
112,모범택시 카카오택시 블랙택시 타세요,2.0
113,역시 엠비씨는 댓글부터 걸러야해 ㅋㅋㅋㅋㅋ,3.0


In [39]:
round(max(model.predict_proba([test_data[1]])[0]), 2)

0.43

In [40]:
df["category"] = df["category_code"].apply(lambda data: categories[data])
df["proba"] = df["content"].apply(
    lambda data: round(max(model.predict_proba([data])[0]), 2) )
df

,content,category_code,category,proba
0,타다 1500대인데 기사가 12000명 이면 차한대에 기사가 10명정도 라는 말인데...,2.0,택시,0.97
1,회사택시하세요 모든것이보장되어있는데 왜타다를선택했나요 뻔히불법이고결과가이럴줄몰랐나요?,2.0,택시,0.43
2,불법콜택시 타다 면허ㆍ자격증 갖고하면 될껄 꽁짜로 쳐묵을려니까 문제지 총량제 ...,2.0,택시,1.00
3,타다 잘 망했다 ~~~ 타다 만세,2.0,택시,0.67
4,국회가 너무 늙었다..\n\n 🙆💬OMG!💨\n 👕\n 👖\n👟👟,3.0,미분류,0.90
...,...,...,...,...
110,공산국가??타다 금지법????,1.0,타다,0.54
111,타다 베이직은 종료하고. 공항이랑 프라이빗렌트 서비스는 합법이니 계속하겠는데요,1.0,타다,0.97
112,모범택시 카카오택시 블랙택시 타세요,2.0,택시,0.93
113,역시 엠비씨는 댓글부터 걸러야해 ㅋㅋㅋㅋㅋ,3.0,미분류,0.95


In [41]:
# 모델 저장
pickle.dump(model, open("clf.pkl", "wb"))